In [1]:
import zipfile
import pandas as pd
import os
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define path to ZIP file in your Google Drive
zip_path = '/content/drive/My Drive/sudoku.csv.zip'
extract_path = '/content/sudoku_data'  # where we'll extract the zip

# Step 3: Unzip the file
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Step 4: Find and read the extracted CSV
csv_file_path = os.path.join(extract_path, 'sudoku.csv')
sudoku_games = pd.read_csv(csv_file_path)

# Step 5: Display dataset info
print(f"\n✅ Dataset Info:")
print(f"Shape: {sudoku_games.shape}")
print(f"Columns: {sudoku_games.columns.tolist()}")
print(f"Memory usage: {sudoku_games.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Step 6: Show first few rows
print(f"\n🧩 First 5 rows:")
sudoku_games.head()

Mounted at /content/drive

✅ Dataset Info:
Shape: (9000000, 2)
Columns: ['puzzle', 'solution']
Memory usage: 2368.93 MB

🧩 First 5 rows:


,puzzle,solution
0,0700000430400096108006349000940520003584600200...,6795182435437296188216349577943521863584617292...
1,3010865040465210705000000014008000020803479000...,3719865248465213795924738614638197522853479167...
2,0483015603600080909106700030200009355090102006...,7483915623652487919126754834217869355894132766...
3,0083170000042051090000400703271609049014500000...,2983176457642851391539462783271689549814537266...
4,0408906300001368208007405190004670524500207002...,1428956379751368248367425193984671524513287962...


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

# Step 7: Process solutions into row dataset (limit to ~11k puzzles for 100k rows)
def create_row_dataset(sudoku_df, max_samples=100000):
    X, y = [], []
    samples_needed = max_samples // 9  # Each puzzle gives 9 rows

    for i, solution in enumerate(sudoku_df['solution']):
        if i >= samples_needed:
            break

        # Process each 9x9 grid row by row
        for row_start in range(0, 81, 9):
            row = [int(d) for d in solution[row_start:row_start+9]]
            input_row = [0] + row[1:]
            target = row[0]

            X.append(input_row)
            y.append(target)

    return np.array(X), np.array(y)

# Create dataset (100k samples)
X, y = create_row_dataset(sudoku_games, 100000)

# Convert to one-hot
X_onehot = np.zeros((len(X), 9, 10))
y_onehot = np.zeros((len(y), 9))

for i, row in enumerate(X):
    for j, num in enumerate(row):
        X_onehot[i, j, num] = 1
    y_onehot[i, y[i]-1] = 1

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_onehot, y_onehot, test_size=0.2, random_state=42
)

print(f"✅ Dataset created: {len(X)} samples")

✅ Dataset created: 99999 samples


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Simple Neural Network Model
model = models.Sequential([
   layers.Flatten(input_shape=(9, 10)),  # Flatten 9x10 to 90
   layers.Dense(128, activation='relu'),
   layers.Dense(64, activation='relu'),
   layers.Dense(9, activation='softmax')  # Output 9 classes (1-9)
])

# Compile
model.compile(
   optimizer='adam',
   loss='categorical_crossentropy',
   metrics=['accuracy']
)

# Train
history = model.fit(
   X_train, y_train,
   epochs=10,
   batch_size=32,
   validation_data=(X_test, y_test),
   verbose=1
)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8269 - loss: 0.5949 - val_accuracy: 1.0000 - val_loss: 3.5716e-04
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 1.0000 - loss: 2.2633e-04 - val_accuracy: 1.0000 - val_loss: 5.1327e-05
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 1.0000 - loss: 3.6188e-05 - val_accuracy: 1.0000 - val_loss: 1.0950e-05
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 1.0000 - loss: 7.9375e-06 - val_accuracy: 1.0000 - val_loss: 2.6038e-06
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 1.0000 - loss: 1.8965e-06 - val_accuracy: 1.0000 - val_loss: 6.2700e-07
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 1.0000 - loss: 4.6373e-07 - val_accuracy: 1.0000 - val_loss: 1.0778e-07
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 1.0000 - loss: 7.8453e-08 - val_accuracy: 1.0000 - val_loss: 9.6261e-09
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Fixed Neural Network with Generalization
model = models.Sequential([
    layers.Input(shape=(9, 10)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),  # Prevent overfitting
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),  # More generalization
    layers.Dense(9, activation='softmax')
])

# Compile
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train with early stopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=3, restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6320 - loss: 1.0828 - val_accuracy: 1.0000 - val_loss: 2.2478e-04
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9998 - loss: 0.0043 - val_accuracy: 1.0000 - val_loss: 1.9379e-05
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9999 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 3.3963e-06
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 1.0000 - loss: 4.5031e-04 - val_accuracy: 1.0000 - val_loss: 8.8472e-07

✅ Test Accuracy: 1.0000


In [4]:
# Test Block - See how model performs
import random

def test_model_predictions(model, X_test, y_test, num_samples=10):
   print("🧪 Testing Model Predictions:\n")

   # Get random test samples
   indices = random.sample(range(len(X_test)), num_samples)

   for i, idx in enumerate(indices):
       # Get input and true answer
       input_row = X_test[idx]
       true_answer = np.argmax(y_test[idx]) + 1  # Convert back to 1-9

       # Convert one-hot back to numbers for display
       display_row = []
       for pos in input_row:
           display_row.append(np.argmax(pos))

       # Make prediction
       prediction = model.predict(input_row.reshape(1, 9, 10), verbose=0)
       predicted_answer = np.argmax(prediction) + 1
       confidence = np.max(prediction) * 100

       # Display result
       print(f"Test {i+1}:")
       print(f"Input:     {display_row}")
       print(f"True:      {true_answer}")
       print(f"Predicted: {predicted_answer} (confidence: {confidence:.1f}%)")
       print(f"{'✅ CORRECT' if predicted_answer == true_answer else '❌ WRONG'}")
       print("-" * 40)

# Run test
test_model_predictions(model, X_test, y_test)

# Overall accuracy
predictions = model.predict(X_test, verbose=0)
predicted_classes = np.argmax(predictions, axis=1) + 1
true_classes = np.argmax(y_test, axis=1) + 1
accuracy = np.mean(predicted_classes == true_classes)
print(f"\n🎯 Overall Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

🧪 Testing Model Predictions:

Test 1:
Input:     [np.int64(0), np.int64(2), np.int64(8), np.int64(9), np.int64(5), np.int64(4), np.int64(6), np.int64(7), np.int64(1)]
True:      3
Predicted: 3 (confidence: 100.0%)
✅ CORRECT
----------------------------------------
Test 2:
Input:     [np.int64(0), np.int64(2), np.int64(6), np.int64(5), np.int64(8), np.int64(7), np.int64(4), np.int64(1), np.int64(3)]
True:      9
Predicted: 9 (confidence: 100.0%)
✅ CORRECT
----------------------------------------
Test 3:
Input:     [np.int64(0), np.int64(2), np.int64(4), np.int64(1), np.int64(3), np.int64(7), np.int64(5), np.int64(8), np.int64(9)]
True:      6
Predicted: 6 (confidence: 100.0%)
✅ CORRECT
----------------------------------------
Test 4:
Input:     [np.int64(0), np.int64(5), np.int64(3), np.int64(2), np.int64(4), np.int64(6), np.int64(1), np.int64(7), np.int64(8)]
True:      9
Predicted: 9 (confidence: 100.0%)
✅ CORRECT
----------------------------------------
Test 5:
Input:     [np.int64(0

In [5]:
# Step 8: Process data for 2 missing numbers per row
def create_2_missing_row_dataset(sudoku_df, max_samples=100000):
   X, y = [], []
   samples_needed = max_samples // 9

   for i, solution in enumerate(sudoku_df['solution']):
       if i >= samples_needed:
           break

       for row_start in range(0, 81, 9):
           row = [int(d) for d in solution[row_start:row_start+9]]

           # Randomly choose 2 positions to make empty
           empty_positions = random.sample(range(9), 2)
           missing_numbers = [row[pos] for pos in empty_positions]

           # Create input with 2 zeros
           input_row = row.copy()
           for pos in empty_positions:
               input_row[pos] = 0

           X.append(input_row)
           y.append(missing_numbers)  # Now we have 2 targets

   return np.array(X), np.array(y)

# Create new dataset
X_2missing, y_2missing = create_2_missing_row_dataset(sudoku_games, 100000)

# Convert to one-hot
X_2missing_onehot = np.zeros((len(X_2missing), 9, 10))
y_2missing_onehot = np.zeros((len(y_2missing), 2, 9))  # 2 outputs, each 9 classes

for i, row in enumerate(X_2missing):
   for j, num in enumerate(row):
       X_2missing_onehot[i, j, num] = 1

   # Two targets
   for k, target in enumerate(y_2missing[i]):
       y_2missing_onehot[i, k, target-1] = 1

# Reshape y for model (flatten 2x9 to 18)
y_2missing_flat = y_2missing_onehot.reshape(len(y_2missing_onehot), 18)

# Train/test split
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
   X_2missing_onehot, y_2missing_flat, test_size=0.2, random_state=42
)

print(f"✅ 2-Missing Dataset: {len(X_2missing)} samples")
print(f"Input shape: {X_train_2.shape}")
print(f"Output shape: {y_train_2.shape}")  # Should be (samples, 18)

✅ 2-Missing Dataset: 99999 samples
Input shape: (79999, 9, 10)
Output shape: (79999, 18)


In [8]:
# New Model for 2 Missing Numbers
model_2missing = models.Sequential([
   layers.Input(shape=(9, 10)),
   layers.Flatten(),
   layers.Dense(256, activation='relu'),  # Bigger since more complex
   layers.Dropout(0.3),
   layers.Dense(128, activation='relu'),
   layers.Dropout(0.2),
   layers.Dense(64, activation='relu'),
   layers.Dense(18, activation='sigmoid')  # 18 outputs (2 numbers × 9 classes each)
])

# Compile with different loss for multi-output
model_2missing.compile(
   optimizer='adam',
   loss='binary_crossentropy',  # Better for multi-label
   metrics=['accuracy']
)

# Train
early_stop = tf.keras.callbacks.EarlyStopping(
   monitor='val_accuracy', patience=5, restore_best_weights=True
)

history_2 = model_2missing.fit(
   X_train_2, y_train_2,
   epochs=25,
   batch_size=64,
   validation_data=(X_test_2, y_test_2),
   callbacks=[early_stop],
   verbose=1
)

# Evaluate
test_loss, test_acc = model_2missing.evaluate(X_test_2, y_test_2, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

Epoch 1/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.1764 - loss: 0.2887 - val_accuracy: 0.2826 - val_loss: 0.1575
Epoch 2/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.2475 - loss: 0.1602 - val_accuracy: 0.2282 - val_loss: 0.1554
Epoch 3/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.2511 - loss: 0.1569 - val_accuracy: 0.2636 - val_loss: 0.1557
Epoch 4/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.2538 - loss: 0.1561 - val_accuracy: 0.2767 - val_loss: 0.1553
Epoch 5/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.2537 - loss: 0.1557 - val_accuracy: 0.2605 - val_loss: 0.1550
Epoch 6/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2491 - loss: 0.1556 - val_accuracy: 0.2176 - val_loss: 0.1547

✅ Test Accuracy: 0.2826


In [12]:
# Test Block for 2 Missing Numbers
def test_2missing_predictions(model, X_test, y_test, num_samples=10):
   print("🧪 Testing 2-Missing Model Predictions:\n")

   indices = random.sample(range(len(X_test)), num_samples)

   for i, idx in enumerate(indices):
       # Get input
       input_row = X_test[idx]

       # Convert one-hot back to numbers for display
       display_row = []
       empty_positions = []
       for pos, one_hot in enumerate(input_row):
           num = np.argmax(one_hot)
           display_row.append(num)
           if num == 0:
               empty_positions.append(pos)

       # Get true answers (reshape from flat 18 back to 2x9)
       true_answers_onehot = y_test[idx].reshape(2, 9)
       true_answers = [np.argmax(true_answers_onehot[0]) + 1,
                      np.argmax(true_answers_onehot[1]) + 1]

       # Make prediction
       prediction = model.predict(input_row.reshape(1, 9, 10), verbose=0)
       pred_reshaped = prediction.reshape(2, 9)

       predicted_answers = [np.argmax(pred_reshaped[0]) + 1,
                          np.argmax(pred_reshaped[1]) + 1]
       confidences = [np.max(pred_reshaped[0]) * 100,
                     np.max(pred_reshaped[1]) * 100]

       # Check if correct
       correct = set(predicted_answers) == set(true_answers)

       print(f"Test {i+1}:")
       print(f"Input:      {display_row}")
       print(f"Empty at:   positions {empty_positions}")
       print(f"True:       {sorted(true_answers)}")
       print(f"Predicted:  {sorted(predicted_answers)} (conf: {confidences[0]:.1f}%, {confidences[1]:.1f}%)")
       print(f"{'✅ CORRECT' if correct else '❌ WRONG'}")
       print("-" * 50)

# Run test
test_2missing_predictions(model_2missing, X_test_2, y_test_2)

# Overall accuracy (both numbers must be correct)
predictions = model_2missing.predict(X_test_2, verbose=0)
pred_reshaped = predictions.reshape(-1, 2, 9)
true_reshaped = y_test_2.reshape(-1, 2, 9)

correct_count = 0
for i in range(len(pred_reshaped)):
   pred_nums = [np.argmax(pred_reshaped[i][0]) + 1, np.argmax(pred_reshaped[i][1]) + 1]
   true_nums = [np.argmax(true_reshaped[i][0]) + 1, np.argmax(true_reshaped[i][1]) + 1]
   if set(pred_nums) == set(true_nums):
       correct_count += 1

accuracy = correct_count / len(pred_reshaped)
print(f"\n🎯 Overall Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

🧪 Testing 2-Missing Model Predictions:

Test 1:
Input:      [np.int64(4), np.int64(0), np.int64(2), np.int64(3), np.int64(1), np.int64(8), np.int64(5), np.int64(0), np.int64(6)]
Empty at:   positions [1, 7]
True:       [np.int64(7), np.int64(9)]
Predicted:  [np.int64(7), np.int64(9)] (conf: 57.2%, 60.6%)
✅ CORRECT
--------------------------------------------------
Test 2:
Input:      [np.int64(3), np.int64(2), np.int64(7), np.int64(1), np.int64(5), np.int64(0), np.int64(8), np.int64(0), np.int64(6)]
Empty at:   positions [5, 7]
True:       [np.int64(4), np.int64(9)]
Predicted:  [np.int64(4), np.int64(4)] (conf: 57.8%, 56.3%)
❌ WRONG
--------------------------------------------------
Test 3:
Input:      [np.int64(0), np.int64(4), np.int64(1), np.int64(5), np.int64(6), np.int64(9), np.int64(0), np.int64(3), np.int64(2)]
Empty at:   positions [0, 6]
True:       [np.int64(7), np.int64(8)]
Predicted:  [np.int64(7), np.int64(7)] (conf: 61.3%, 54.0%)
❌ WRONG
----------------------------------

✅ Goal
For each sample:

Input = 2 Sudoku rows (each with 1 digit missing, i.e., [0, d2, ..., d9])

Target = the missing digit of the first row

In [17]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

def create_two_row_dataset(sudoku_df, max_samples=100000):
    X, y = [], []
    samples_needed = max_samples // 36  # Each puzzle generates 36 samples (9 rows × 4 pairs each)

    for i, solution in enumerate(sudoku_df['solution']):
        if i >= samples_needed:
            break

        # Convert solution to 9x9 grid
        rows = []
        for row_start in range(0, 81, 9):
            row = [int(d) for d in solution[row_start:row_start+9]]
            rows.append(row)

        # Create pairs from consecutive rows
        for j in range(8):  # rows 0-1, 1-2, ..., 7-8
            row1 = rows[j].copy()
            row2 = rows[j + 1].copy()

            # Generate multiple samples per row pair
            for _ in range(4):  # 4 samples per pair
                # Hide random positions
                pos1 = random.randint(0, 8)
                pos2 = random.randint(0, 8)

                # Store original values
                missing1 = row1[pos1]
                missing2 = row2[pos2]

                # Create input with hidden digits
                input_row1 = row1.copy()
                input_row2 = row2.copy()
                input_row1[pos1] = 0
                input_row2[pos2] = 0

                # Combine both rows (18 elements)
                combined_input = input_row1 + input_row2

                # Target: both missing digits
                target = [missing1, missing2]

                X.append(combined_input)
                y.append(target)

    return np.array(X), np.array(y)

# Create dataset
print("🔄 Creating two-row dataset...")
X, y = create_two_row_dataset(sudoku_games, max_samples=50000)
print(f"Dataset shape: X={X.shape}, y={y.shape}")

# One-hot encode inputs (18 positions, 10 classes each: 0-9)
X_onehot = np.zeros((len(X), 18, 10))
for i, row in enumerate(X):
    for j, num in enumerate(row):
        X_onehot[i, j, num] = 1

# One-hot encode targets (2 digits, 9 classes each: 1-9)
y_onehot = np.zeros((len(y), 2, 9))
for i, targets in enumerate(y):
    y_onehot[i, 0, targets[0] - 1] = 1  # First missing digit (1-9 → 0-8)
    y_onehot[i, 1, targets[1] - 1] = 1  # Second missing digit (1-9 → 0-8)

# Reshape y for model training (flatten to 18 outputs)
y_flat = y_onehot.reshape(len(y_onehot), 18)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_onehot, y_flat, test_size=0.2, random_state=42
)

print(f"✅ Training data: X={X_train.shape}, y={y_train.shape}")
print(f"✅ Test data: X={X_test.shape}, y={y_test.shape}")
print(f"Sample input shape: {X_train[0].shape}")
print(f"Sample target shape: {y_train[0].shape}")

🔄 Creating two-row dataset...
Dataset shape: X=(44416, 18), y=(44416, 2)
✅ Training data: X=(35532, 18, 10), y=(35532, 18)
✅ Test data: X=(8884, 18, 10), y=(8884, 18)
Sample input shape: (18, 10)
Sample target shape: (18,)


In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Neural Network for Two Sudoku Rows (predicts 2 missing digits)
model = models.Sequential([
    layers.Input(shape=(18, 10)),           # Input: 2 rows × 9 digits (each one-hot of size 10)
    layers.Flatten(),                       # Flatten 18x10 = 180 features
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(18, activation='sigmoid')  # Output: 18 values (2 rows × 9 digits each)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Changed from categorical_crossentropy
    metrics=['accuracy']
)

# Early stopping to prevent overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Train the model
print("🚀 Training model...")
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

# Evaluate model performance
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

print("\n📊 Model Summary:")
model.summary()

🚀 Training model...
Epoch 1/20
556/556 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.1489 - loss: 0.3494 - val_accuracy: 0.4972 - val_loss: 0.0219
Epoch 2/20
556/556 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.5039 - loss: 0.0307 - val_accuracy: 0.5129 - val_loss: 6.0640e-04
Epoch 3/20
556/556 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4944 - loss: 0.0053 - val_accuracy: 0.4685 - val_loss: 1.2175e-04
Epoch 4/20
556/556 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4948 - loss: 0.0024 - val_accuracy: 0.5084 - val_loss: 4.6207e-05
Epoch 5/20
556/556 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4947 - loss: 0.0016 - val_accuracy: 0.4496 - val_loss: 1.4672e-05

✅ Test Accuracy: 0.5129

📊 Model Summary:


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_4 (Flatten)             │ (None, 180)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │        46,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 18)             │         2,322 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 244,664 (955.72 KB)

 Trainable params: 81,554 (318.57 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 163,110 (637.15 KB)

In [20]:
# Test Block - See dual predictions
import random
import numpy as np

def test_dual_predictions(model, X_test, y_test, num_samples=10):
    print("🧪 Testing Dual Prediction Model:\n")

    indices = random.sample(range(len(X_test)), num_samples)

    for i, idx in enumerate(indices):
        # Get input and true answers
        input_rows = X_test[idx]  # Shape: (18, 10)
        true_targets = y_test[idx]  # Shape: (18,) - flattened

        # Reshape true targets back to (2, 9)
        true_reshaped = true_targets.reshape(2, 9)
        true_digit1 = np.argmax(true_reshaped[0]) + 1  # First row missing digit
        true_digit2 = np.argmax(true_reshaped[1]) + 1  # Second row missing digit

        # Convert input back to display format
        row1 = [np.argmax(input_rows[j]) for j in range(9)]
        row2 = [np.argmax(input_rows[j]) for j in range(9, 18)]

        # Find missing positions (where digit is 0)
        missing_pos1 = row1.index(0) if 0 in row1 else -1
        missing_pos2 = row2.index(0) if 0 in row2 else -1

        # Make prediction
        prediction = model.predict(input_rows.reshape(1, 18, 10), verbose=0)
        pred_reshaped = prediction.reshape(2, 9)

        # Get predicted digits
        pred_digit1 = np.argmax(pred_reshaped[0]) + 1
        pred_digit2 = np.argmax(pred_reshaped[1]) + 1

        # Get confidence scores
        conf1 = np.max(pred_reshaped[0]) * 100
        conf2 = np.max(pred_reshaped[1]) * 100

        # Display results
        print(f"Test {i+1}:")
        print(f"Row 1:     {row1} (missing pos {missing_pos1})")
        print(f"Row 2:     {row2} (missing pos {missing_pos2})")
        print(f"True:      [{true_digit1}, {true_digit2}]")
        print(f"Predicted: [{pred_digit1}, {pred_digit2}]")
        print(f"Confidence: [{conf1:.1f}%, {conf2:.1f}%]")

        # Check correctness
        correct1 = pred_digit1 == true_digit1
        correct2 = pred_digit2 == true_digit2
        both_correct = correct1 and correct2

        print(f"Row 1: {'✅' if correct1 else '❌'} | Row 2: {'✅' if correct2 else '❌'} | Both: {'✅' if both_correct else '❌'}")
        print("-" * 50)

# Run test
test_dual_predictions(model, X_test, y_test)

# Overall accuracy for dual predictions
print("\n🎯 Overall Accuracy Analysis:")
predictions = model.predict(X_test, verbose=0)
pred_reshaped = predictions.reshape(-1, 2, 9)
true_reshaped = y_test.reshape(-1, 2, 9)

# Individual row accuracies
row1_pred = np.argmax(pred_reshaped[:, 0, :], axis=1) + 1
row1_true = np.argmax(true_reshaped[:, 0, :], axis=1) + 1
row1_acc = np.mean(row1_pred == row1_true)

row2_pred = np.argmax(pred_reshaped[:, 1, :], axis=1) + 1
row2_true = np.argmax(true_reshaped[:, 1, :], axis=1) + 1
row2_acc = np.mean(row2_pred == row2_true)

# Both correct accuracy
both_correct = (row1_pred == row1_true) & (row2_pred == row2_true)
both_acc = np.mean(both_correct)

print(f"Row 1 Accuracy: {row1_acc:.4f} ({row1_acc*100:.2f}%)")
print(f"Row 2 Accuracy: {row2_acc:.4f} ({row2_acc*100:.2f}%)")
print(f"Both Correct:   {both_acc:.4f} ({both_acc*100:.2f}%)")
print(f"Average:        {(row1_acc + row2_acc)/2:.4f} ({(row1_acc + row2_acc)*50:.2f}%)")

🧪 Testing Dual Prediction Model:

Test 1:
Row 1:     [np.int64(4), np.int64(6), np.int64(9), np.int64(5), np.int64(3), np.int64(2), np.int64(1), np.int64(0), np.int64(7)] (missing pos 7)
Row 2:     [np.int64(7), np.int64(8), np.int64(3), np.int64(1), np.int64(4), np.int64(9), np.int64(0), np.int64(2), np.int64(6)] (missing pos 6)
True:      [8, 5]
Predicted: [8, 5]
Confidence: [99.7%, 99.8%]
Row 1: ✅ | Row 2: ✅ | Both: ✅
--------------------------------------------------
Test 2:
Row 1:     [np.int64(5), np.int64(4), np.int64(7), np.int64(3), np.int64(2), np.int64(6), np.int64(0), np.int64(8), np.int64(1)] (missing pos 6)
Row 2:     [np.int64(6), np.int64(9), np.int64(1), np.int64(7), np.int64(5), np.int64(0), np.int64(2), np.int64(3), np.int64(4)] (missing pos 5)
True:      [9, 8]
Predicted: [9, 8]
Confidence: [99.8%, 99.9%]
Row 1: ✅ | Row 2: ✅ | Both: ✅
--------------------------------------------------
Test 3:
Row 1:     [np.int64(6), np.int64(0), np.int64(1), np.int64(4), np.int64(2

Single row input: 9 positions (2 missing, 7 filled)
2 random positions hidden per sample
Dual targets: Predicts both missing digits

In [26]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

def create_single_row_dataset(sudoku_df, max_samples=200000):
    """
    Create dataset where 2 digits are missing from a single row
    Input: 9 positions (2 are 0, 7 are filled)
    Output: 2 missing digits
    """
    X, y = [], []
    samples_needed = max_samples // 45  # Each puzzle generates ~45 samples (9 rows × 5 pairs each)

    for i, solution in enumerate(sudoku_df['solution']):
        if i >= samples_needed:
            break

        # Convert solution to 9x9 grid
        rows = []
        for row_start in range(0, 81, 9):
            row = [int(d) for d in solution[row_start:row_start+9]]
            rows.append(row)

        # Process each row
        for row in rows:
            # Generate multiple samples per row (different position pairs)
            for _ in range(5):  # 5 samples per row
                # Hide 2 random positions
                positions = random.sample(range(9), 2)  # Pick 2 different positions
                pos1, pos2 = sorted(positions)  # Sort for consistency

                # Store original values
                missing1 = row[pos1]
                missing2 = row[pos2]

                # Create input with 2 hidden digits
                input_row = row.copy()
                input_row[pos1] = 0
                input_row[pos2] = 0

                # Target: both missing digits
                target = [missing1, missing2]

                X.append(input_row)
                y.append(target)

    return np.array(X), np.array(y)

# Create dataset
print("🔄 Creating single-row dataset (2 missing digits)...")
X, y = create_single_row_dataset(sudoku_games, max_samples=50000)
print(f"Dataset shape: X={X.shape}, y={y.shape}")

# One-hot encode inputs (9 positions, 10 classes each: 0-9)
X_onehot = np.zeros((len(X), 9, 10))
for i, row in enumerate(X):
    for j, num in enumerate(row):
        X_onehot[i, j, num] = 1

# One-hot encode targets (2 digits, 9 classes each: 1-9)
y_onehot = np.zeros((len(y), 2, 9))
for i, targets in enumerate(y):
    y_onehot[i, 0, targets[0] - 1] = 1  # First missing digit (1-9 → 0-8)
    y_onehot[i, 1, targets[1] - 1] = 1  # Second missing digit (1-9 → 0-8)

# Reshape y for model training (flatten to 18 outputs)
y_flat = y_onehot.reshape(len(y_onehot), 18)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_onehot, y_flat, test_size=0.2, random_state=42
)

print(f"✅ Training data: X={X_train.shape}, y={y_train.shape}")
print(f"✅ Test data: X={X_test.shape}, y={y_test.shape}")
print(f"Sample input: {X_train[0].shape} (9 positions × 10 classes)")
print(f"Sample target: {y_train[0].shape} (18 outputs: 2 digits × 9 classes)")

# Display sample
sample_input = np.argmax(X_train[0], axis=1)
sample_target = y_train[0].reshape(2, 9)
target1 = np.argmax(sample_target[0]) + 1
target2 = np.argmax(sample_target[1]) + 1
print(f"\nSample:")
print(f"Input row: {sample_input.tolist()} (0s are missing)")
print(f"Missing digits: [{target1}, {target2}]")

🔄 Creating single-row dataset (2 missing digits)...
Dataset shape: X=(49995, 9), y=(49995, 2)
✅ Training data: X=(39996, 9, 10), y=(39996, 18)
✅ Test data: X=(9999, 9, 10), y=(9999, 18)
Sample input: (9, 10) (9 positions × 10 classes)
Sample target: (18,) (18 outputs: 2 digits × 9 classes)

Sample:
Input row: [2, 7, 3, 4, 9, 8, 0, 1, 0] (0s are missing)
Missing digits: [5, 6]


In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Neural Network for Single Sudoku Row (predicts 2 missing digits)
model = models.Sequential([
    layers.Input(shape=(9, 10)),            # Input: 1 row × 9 digits (each one-hot of size 10)
    layers.Flatten(),                       # Flatten 9x10 = 90 features
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(18, activation='sigmoid')  # Output: 18 values (2 digits × 9 classes each)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping to prevent overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Train the model
print("🚀 Training single-row dual prediction model...")
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

# Evaluate model performance
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

print("\n📊 Model Summary:")
model.summary()

🚀 Training single-row dual prediction model...
Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.0903 - loss: 0.3894 - val_accuracy: 0.2726 - val_loss: 0.1739
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2457 - loss: 0.1918 - val_accuracy: 0.2315 - val_loss: 0.1603
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2607 - loss: 0.1749 - val_accuracy: 0.2761 - val_loss: 0.1572
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2605 - loss: 0.1666 - val_accuracy: 0.2965 - val_loss: 0.1554
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2640 - loss: 0.1612 - val_accuracy: 0.2467 - val_loss: 0.1547
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2691 - loss: 0.1583 - val_accuracy: 0.2501 - val_loss: 0.1547
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.2653 - loss: 0.1569 - val_accuracy: 0.2246 - val_loss: 0.1545

✅ Test Accuracy: 0.2965

📊 Model Summary:


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_7 (Flatten)             │ (None, 90)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 128)            │        11,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 18)             │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 63,224 (246.97 KB)

 Trainable params: 21,074 (82.32 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 42,150 (164.65 KB)

In [29]:
# Test Block - Single Row Dual Predictions (10 cases)
import random
import numpy as np

def test_single_row_predictions(model, X_test, y_test, num_samples=10):
    print("🧪 Testing Single Row Dual Prediction Model:\n")

    indices = random.sample(range(len(X_test)), num_samples)

    for i, idx in enumerate(indices):
        # Get input and true answers
        input_row = X_test[idx]  # Shape: (9, 10)
        true_targets = y_test[idx]  # Shape: (18,) - flattened

        # Reshape true targets back to (2, 9)
        true_reshaped = true_targets.reshape(2, 9)
        true_digit1 = np.argmax(true_reshaped[0]) + 1  # First missing digit
        true_digit2 = np.argmax(true_reshaped[1]) + 1  # Second missing digit

        # Convert input back to display format
        row = [np.argmax(input_row[j]) for j in range(9)]

        # Find missing positions (where digit is 0)
        missing_positions = [pos for pos, digit in enumerate(row) if digit == 0]

        # Make prediction
        prediction = model.predict(input_row.reshape(1, 9, 10), verbose=0)
        pred_reshaped = prediction.reshape(2, 9)

        # Get predicted digits
        pred_digit1 = np.argmax(pred_reshaped[0]) + 1
        pred_digit2 = np.argmax(pred_reshaped[1]) + 1

        # Get confidence scores
        conf1 = np.max(pred_reshaped[0]) * 100
        conf2 = np.max(pred_reshaped[1]) * 100

        # Display results
        print(f"Test {i+1}:")
        print(f"Row:       {row}")
        print(f"Missing:   Positions {missing_positions}")
        print(f"True:      [{true_digit1}, {true_digit2}]")
        print(f"Predicted: [{pred_digit1}, {pred_digit2}]")
        print(f"Confidence: [{conf1:.1f}%, {conf2:.1f}%]")

        # Check correctness
        correct1 = pred_digit1 == true_digit1
        correct2 = pred_digit2 == true_digit2
        both_correct = correct1 and correct2

        print(f"Digit 1: {'✅' if correct1 else '❌'} | Digit 2: {'✅' if correct2 else '❌'} | Both: {'✅' if both_correct else '❌'}")
        print("-" * 60)

# Run test
test_single_row_predictions(model, X_test, y_test)

# Overall accuracy for dual predictions
print("\n🎯 Overall Accuracy Analysis:")
predictions = model.predict(X_test, verbose=0)
pred_reshaped = predictions.reshape(-1, 2, 9)
true_reshaped = y_test.reshape(-1, 2, 9)

# Individual digit accuracies
digit1_pred = np.argmax(pred_reshaped[:, 0, :], axis=1) + 1
digit1_true = np.argmax(true_reshaped[:, 0, :], axis=1) + 1
digit1_acc = np.mean(digit1_pred == digit1_true)

digit2_pred = np.argmax(pred_reshaped[:, 1, :], axis=1) + 1
digit2_true = np.argmax(true_reshaped[:, 1, :], axis=1) + 1
digit2_acc = np.mean(digit2_pred == digit2_true)

# Both correct accuracy
both_correct = (digit1_pred == digit1_true) & (digit2_pred == digit2_true)
both_acc = np.mean(both_correct)

print(f"First Digit Accuracy:  {digit1_acc:.4f} ({digit1_acc*100:.2f}%)")
print(f"Second Digit Accuracy: {digit2_acc:.4f} ({digit2_acc*100:.2f}%)")
print(f"Both Correct:          {both_acc:.4f} ({both_acc*100:.2f}%)")
print(f"Average Accuracy:      {(digit1_acc + digit2_acc)/2:.4f} ({(digit1_acc + digit2_acc)*50:.2f}%)")

# Additional statistics
total_samples = len(X_test)
correct_both = np.sum(both_correct)
correct_at_least_one = np.sum((digit1_pred == digit1_true) | (digit2_pred == digit2_true))

print(f"\n📊 Additional Stats:")
print(f"Total test samples: {total_samples}")
print(f"Both digits correct: {correct_both} ({correct_both/total_samples*100:.1f}%)")
print(f"At least one correct: {correct_at_least_one} ({correct_at_least_one/total_samples*100:.1f}%)")
print(f"Both digits wrong: {total_samples - correct_at_least_one} ({(total_samples - correct_at_least_one)/total_samples*100:.1f}%)")

🧪 Testing Single Row Dual Prediction Model:

Test 1:
Row:       [np.int64(5), np.int64(1), np.int64(3), np.int64(0), np.int64(7), np.int64(0), np.int64(8), np.int64(9), np.int64(4)]
Missing:   Positions [3, 5]
True:      [6, 2]
Predicted: [2, 2]
Confidence: [51.5%, 54.5%]
Digit 1: ❌ | Digit 2: ✅ | Both: ❌
------------------------------------------------------------
Test 2:
Row:       [np.int64(8), np.int64(3), np.int64(0), np.int64(6), np.int64(7), np.int64(9), np.int64(0), np.int64(1), np.int64(2)]
Missing:   Positions [2, 6]
True:      [5, 4]
Predicted: [5, 5]
Confidence: [52.3%, 49.5%]
Digit 1: ✅ | Digit 2: ❌ | Both: ❌
------------------------------------------------------------
Test 3:
Row:       [np.int64(0), np.int64(0), np.int64(4), np.int64(2), np.int64(9), np.int64(8), np.int64(1), np.int64(7), np.int64(5)]
Missing:   Positions [0, 1]
True:      [6, 3]
Predicted: [6, 3]
Confidence: [47.9%, 51.1%]
Digit 1: ✅ | Digit 2: ✅ | Both: ✅
-----------------------------------------------

In [31]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

def create_reverse_l_dataset(sudoku_df, max_samples=200000):
    """
    Create dataset where 2 digits are missing in reverse L pattern:
    - One horizontal row (9 positions) with intersection missing
    - One vertical column (9 positions) with intersection missing
    - Second missing position is either in the row OR column
    Input: horizontal_row (9 values) + vertical_col (9 values) = 18 total inputs
    Output: 2 missing digits (intersection + second position)
    """
    X_horizontal, X_vertical, y = [], [], []
    samples_needed = max_samples // 81  # Each puzzle generates ~81 samples (9×9 intersection points)

    for i, solution in enumerate(sudoku_df['solution']):
        if i >= samples_needed:
            break

        # Convert solution to 9x9 grid
        grid = []
        for row_start in range(0, 81, 9):
            row = [int(d) for d in solution[row_start:row_start+9]]
            grid.append(row)

        # Try different intersection points
        for intersection_row in range(9):
            for intersection_col in range(9):
                # Generate 2 samples per intersection (one with row missing, one with col missing)
                for sample_type in ['horizontal_extra', 'vertical_extra']:

                    # Get horizontal row and vertical column
                    horizontal_row = grid[intersection_row].copy()
                    vertical_col = [grid[r][intersection_col] for r in range(9)]

                    # Store intersection value
                    intersection_value = grid[intersection_row][intersection_col]

                    # Choose second missing position
                    if sample_type == 'horizontal_extra':
                        # Second missing is somewhere else in the horizontal row
                        available_cols = [c for c in range(9) if c != intersection_col]
                        second_missing_col = random.choice(available_cols)
                        second_missing_value = horizontal_row[second_missing_col]

                        # Hide both positions in horizontal row
                        horizontal_row[intersection_col] = 0
                        horizontal_row[second_missing_col] = 0

                        # Hide only intersection in vertical column
                        vertical_col[intersection_row] = 0

                        target = [intersection_value, second_missing_value]

                    else:  # vertical_extra
                        # Second missing is somewhere else in the vertical column
                        available_rows = [r for r in range(9) if r != intersection_row]
                        second_missing_row = random.choice(available_rows)
                        second_missing_value = vertical_col[second_missing_row]

                        # Hide only intersection in horizontal row
                        horizontal_row[intersection_col] = 0

                        # Hide both positions in vertical column
                        vertical_col[intersection_row] = 0
                        vertical_col[second_missing_row] = 0

                        target = [intersection_value, second_missing_value]

                    X_horizontal.append(horizontal_row)
                    X_vertical.append(vertical_col)
                    y.append(target)

    return np.array(X_horizontal), np.array(X_vertical), np.array(y)

# Create dataset
print("🔄 Creating reverse L dataset (2 missing digits)...")
X_horizontal, X_vertical, y = create_reverse_l_dataset(sudoku_games, max_samples=50000)
print(f"Dataset shape: X_horizontal={X_horizontal.shape}, X_vertical={X_vertical.shape}, y={y.shape}")

# One-hot encode horizontal inputs (9 positions, 10 classes each: 0-9)
X_horizontal_onehot = np.zeros((len(X_horizontal), 9, 10))
for i, row in enumerate(X_horizontal):
    for j, num in enumerate(row):
        X_horizontal_onehot[i, j, num] = 1

# One-hot encode vertical inputs (9 positions, 10 classes each: 0-9)
X_vertical_onehot = np.zeros((len(X_vertical), 9, 10))
for i, col in enumerate(X_vertical):
    for j, num in enumerate(col):
        X_vertical_onehot[i, j, num] = 1

# Combine horizontal and vertical inputs
X_combined = np.concatenate([X_horizontal_onehot, X_vertical_onehot], axis=1)  # Shape: (samples, 18, 10)

# One-hot encode targets (2 digits, 9 classes each: 1-9)
y_onehot = np.zeros((len(y), 2, 9))
for i, targets in enumerate(y):
    y_onehot[i, 0, targets[0] - 1] = 1  # First missing digit (1-9 → 0-8)
    y_onehot[i, 1, targets[1] - 1] = 1  # Second missing digit (1-9 → 0-8)

# Reshape y for model training (flatten to 18 outputs)
y_flat = y_onehot.reshape(len(y_onehot), 18)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_flat, test_size=0.2, random_state=42
)

print(f"✅ Training data: X={X_train.shape}, y={y_train.shape}")
print(f"✅ Test data: X={X_test.shape}, y={y_test.shape}")
print(f"Sample input: {X_train[0].shape} (18 positions × 10 classes: 9 horizontal + 9 vertical)")
print(f"Sample target: {y_train[0].shape} (18 outputs: 2 digits × 9 classes)")

# Display sample
sample_horizontal = np.argmax(X_train[0][:9], axis=1)  # First 9 positions
sample_vertical = np.argmax(X_train[0][9:], axis=1)   # Next 9 positions
sample_target = y_train[0].reshape(2, 9)
target1 = np.argmax(sample_target[0]) + 1
target2 = np.argmax(sample_target[1]) + 1

print(f"\nSample:")
print(f"Horizontal row: {sample_horizontal.tolist()} (0s are missing)")
print(f"Vertical col:   {sample_vertical.tolist()} (0s are missing)")
print(f"Missing digits: [{target1}, {target2}] (intersection + second position)")
print(f"Input shape: {X_train[0].shape} → Horizontal(9×10) + Vertical(9×10) = 18×10")

🔄 Creating reverse L dataset (2 missing digits)...
Dataset shape: X_horizontal=(99954, 9), X_vertical=(99954, 9), y=(99954, 2)
✅ Training data: X=(79963, 18, 10), y=(79963, 18)
✅ Test data: X=(19991, 18, 10), y=(19991, 18)
Sample input: (18, 10) (18 positions × 10 classes: 9 horizontal + 9 vertical)
Sample target: (18,) (18 outputs: 2 digits × 9 classes)

Sample:
Horizontal row: [2, 4, 0, 8, 6, 9, 7, 1, 3] (0s are missing)
Vertical col:   [0, 1, 7, 3, 0, 4, 2, 6, 9] (0s are missing)
Missing digits: [5, 8] (intersection + second position)
Input shape: (18, 10) → Horizontal(9×10) + Vertical(9×10) = 18×10


In [32]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Neural Network for Reverse L Pattern (predicts 2 missing digits)
model = models.Sequential([
    layers.Input(shape=(18, 10)),           # Input: 18 positions × 10 classes (9 horizontal + 9 vertical)
    layers.Flatten(),                       # Flatten 18x10 = 180 features
    layers.Dense(256, activation='relu'),   # Larger layer for more complex pattern
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(18, activation='sigmoid')  # Output: 18 values (2 digits × 9 classes each)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Train the model
print("🚀 Training reverse L dual prediction model...")
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

print("\n📊 Model Summary:")
model.summary()

🚀 Training reverse L dual prediction model...
Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.5530 - loss: 0.2597 - val_accuracy: 0.9955 - val_loss: 0.0013
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9518 - loss: 0.0075 - val_accuracy: 0.9957 - val_loss: 9.7685e-05
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9284 - loss: 0.0029 - val_accuracy: 0.9870 - val_loss: 2.3187e-05
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9091 - loss: 0.0025 - val_accuracy: 0.9852 - val_loss: 7.1046e-06
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9115 - loss: 0.0021 - val_accuracy: 0.9713 - val_loss: 5.1108e-06

✅ Test Accuracy: 0.9957

📊 Model Summary:


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_8 (Flatten)             │ (None, 180)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 256)            │        46,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 18)             │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 265,976 (1.01 MB)

 Trainable params: 88,658 (346.32 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 177,318 (692.65 KB)

In [33]:
import numpy as np

# Test the trained model with sample predictions
print("🧪 Testing Reverse L Model with 10 cases...")
print("=" * 60)

# Get 10 test samples
test_samples = X_test[:10]
true_labels = y_test[:10]

# Make predictions
predictions = model.predict(test_samples, verbose=0)

# Test each case
for i in range(10):
    print(f"\n📋 Test Case {i+1}:")

    # Decode input (horizontal + vertical)
    horizontal = np.argmax(test_samples[i][:9], axis=1)
    vertical = np.argmax(test_samples[i][9:], axis=1)

    # Decode true labels
    true_reshaped = true_labels[i].reshape(2, 9)
    true_digit1 = np.argmax(true_reshaped[0]) + 1
    true_digit2 = np.argmax(true_reshaped[1]) + 1

    # Decode predictions
    pred_reshaped = predictions[i].reshape(2, 9)
    pred_digit1 = np.argmax(pred_reshaped[0]) + 1
    pred_digit2 = np.argmax(pred_reshaped[1]) + 1

    # Calculate confidence
    confidence1 = np.max(pred_reshaped[0]) * 100
    confidence2 = np.max(pred_reshaped[1]) * 100

    # Check accuracy
    correct1 = "✅" if pred_digit1 == true_digit1 else "❌"
    correct2 = "✅" if pred_digit2 == true_digit2 else "❌"

    print(f"  Horizontal: {horizontal.tolist()} (0s = missing)")
    print(f"  Vertical:   {vertical.tolist()} (0s = missing)")
    print(f"  True:       [{true_digit1}, {true_digit2}]")
    print(f"  Predicted:  [{pred_digit1}, {pred_digit2}] {correct1}{correct2}")
    print(f"  Confidence: [{confidence1:.1f}%, {confidence2:.1f}%]")

# Overall accuracy
predictions_reshaped = predictions.reshape(-1, 2, 9)
true_reshaped = true_labels.reshape(-1, 2, 9)

pred_digits = np.argmax(predictions_reshaped, axis=2) + 1
true_digits = np.argmax(true_reshaped, axis=2) + 1

accuracy = np.mean(pred_digits == true_digits) * 100

print(f"\n🎯 Overall Test Accuracy: {accuracy:.1f}%")
print(f"📊 Correct predictions: {np.sum(pred_digits == true_digits)}/{len(pred_digits.flatten())}")

🧪 Testing Reverse L Model with 10 cases...

📋 Test Case 1:
  Horizontal: [9, 3, 2, 1, 0, 8, 4, 7, 5] (0s = missing)
  Vertical:   [1, 9, 5, 0, 4, 0, 8, 3, 2] (0s = missing)
  True:       [6, 7]
  Predicted:  [6, 7] ✅✅
  Confidence: [100.0%, 100.0%]

📋 Test Case 2:
  Horizontal: [2, 1, 7, 0, 4, 8, 5, 6, 9] (0s = missing)
  Vertical:   [0, 9, 4, 2, 5, 0, 7, 1, 8] (0s = missing)
  True:       [3, 6]
  Predicted:  [3, 6] ✅✅
  Confidence: [100.0%, 99.9%]

📋 Test Case 3:
  Horizontal: [9, 7, 0, 0, 4, 2, 8, 5, 1] (0s = missing)
  Vertical:   [7, 2, 5, 8, 4, 9, 0, 1, 6] (0s = missing)
  True:       [3, 6]
  Predicted:  [3, 6] ✅✅
  Confidence: [100.0%, 100.0%]

📋 Test Case 4:
  Horizontal: [6, 5, 7, 0, 4, 2, 1, 3, 9] (0s = missing)
  Vertical:   [1, 4, 3, 9, 0, 7, 6, 5, 0] (0s = missing)
  True:       [8, 2]
  Predicted:  [8, 2] ✅✅
  Confidence: [100.0%, 99.8%]

📋 Test Case 5:
  Horizontal: [8, 4, 1, 9, 6, 0, 0, 3, 2] (0s = missing)
  Vertical:   [1, 5, 3, 2, 9, 4, 6, 8, 0] (0s = missing)
  Tru